In [4]:
%load_ext autoreload
%autoreload 2
import rootutils
import os
from pathlib import Path

root = rootutils.setup_root(
    os.path.abspath(""), indicator=".project-root", pythonpath=True
)

import time

import src.data.components.processed_tile_group_pb2 as pbf
import pandas as pd
import plotly.express as px
import umap.umap_ as umap
from src.modules.tag_encoder import TagEncoder
from src.data.tiles_datamodule import TilesDataModule

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
tile_group = pbf.TileGroup()
working_dir = os.getcwd()
print(os.getcwd())
with open(root / Path("data/tiles/sthlm/processed/14_9006_11577.pbf"), "rb") as f:
    tile_group.ParseFromString(f.read())

/Users/theo/Documents/courses/current/master-thesis/geojepa/notebooks


In [6]:
start = time.time()
encoder = TagEncoder(embedding_file=root / Path("data/tiles/embeddings.pkl"))
print(f"Time to load encoder: {time.time() - start} s")

Time to load encoder: 0.5928351879119873 s


In [27]:

%autoreload 2
from src.modules.tag_encoder import TagEncoder
from src.modules.geometry_encoder import GeometryEncoder

dataset = TilesDataModule(root / Path("data/tiles/sthlm/processed"), num_workers=0)
dataset.setup()
loader = dataset.test_dataloader()
tiles = 0

encoder = TagEncoder(embedding_file=root / Path("data/tiles/embeddings.pkl"))

geo_encoder = GeometryEncoder()

start_time = time.time()
for i, batch in enumerate(loader):
    print("BATCH", i)
    tiles += batch.size
    embs, strings = encoder(batch.tags)
    print("NBR_FEAT:", batch.nbr_features)
    print("TAGS:", embs.shape)
    geo_embs = geo_encoder(
        batch.nodes, batch.intra_edges, batch.inter_edges, batch.node_to_feature
    )
    print("GEOS:", geo_embs.shape)
    break

end_time = time.time()
print("Time: ", (end_time - start_time) * 1e3, "ms")
print("Time: ", (end_time - start_time) * 1e3 / tiles, "ms per tile")
print("Tiles: ", tiles)


def flatten(matrix):
    return [item for row in matrix for item in row]

tensor([   0,    1,    2,  ..., 1003, 1004, 1005])
BATCH 0
NBR_FEAT: 1006
TAGS: torch.Size([32, 252, 1024])
GEOS: torch.Size([1006, 2048])
torch.Size([1006]) tensor(1005)
Time:  447.49999046325684 ms
Time:  13.984374701976776 ms per tile
Tiles:  32


In [5]:
# df = pd.DataFrame(np.nan, index=range(len(embeddings_array)), columns=range(3))
df = pd.DataFrame({"matching_string": tag_strings})
# Apply UMAP to reduce to 2D
umap_model = umap.UMAP(n_components=2, random_state=42, n_neighbors=60)
umap_results = umap_model.fit_transform(tag_embeddings)

# Add UMAP results to DataFrame
df["umap-2d-one"] = umap_results[:, 0]
df["umap-2d-two"] = umap_results[:, 1]

# Create an interactive Plotly scatter plot
fig = px.scatter(df, x="umap-2d-one", y="umap-2d-two", hover_name="matching_string")

fig.write_html("plots/" + "features-speed" + ".html")

/opt/homebrew/anaconda3/envs/geojepa-ipynb/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [6]:
%autoreload 2
from src.datasets.tiles import TilesDataModule

dataset = TilesDataModule("../../../osm_pipeline/tiles/sthlm/processed", num_workers=0)
dataset.setup()

ModuleNotFoundError: No module named 'src.datasets'

In [ ]:
# df = pd.DataFrame(np.nan, index=range(len(embeddings_array)), columns=range(3))
# Apply UMAP to reduce to 2D
umap_model = umap.UMAP(n_components=2, random_state=42)

loader = dataset.train_dataloader()
tag_embeddings = []
tag_strings = []
for i, batch in enumerate(loader):
    embs, strings = encoder(batch.tags)
    for j, s in enumerate(strings):
        tag_strings.append(s)
        tag_embeddings.append(embs[j].numpy())

umap_results = umap_model.fit_transform(tag_embeddings)

df = pd.DataFrame({"matching_string": tag_strings})
# Add UMAP results to DataFrame
df["umap-2d-one"] = umap_results[:, 0]
df["umap-2d-two"] = umap_results[:, 1]

# Create an interactive Plotly scatter plot
fig = px.scatter(df, x="umap-2d-one", y="umap-2d-two", hover_name="matching_string")

fig.write_html("plots/" + "tiles" + ".html")